In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

In [2]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [3]:
var = "dlyfrzthw"

In [4]:
#Variable 
variable85="rcp85_"+var+"_monthly"
variable45="rcp45_"+var+"_monthly"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_dlyfrzthw_monthly.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_dlyfrzthw_monthly.nc

In [5]:
monthlist = ["january", "february", "march", "april", "may", "june", "july", "august", "september", 
             "october", "november", "december"]

# FOR RCP 45

## Subsetting arrays by periods of time with Xclim

In [6]:
dsEns45= ens.create_ensemble(files45)
dsEns45.time.dt.month[0:12]
# Monthly example 30 y means
listds = []
i=0
for s in dsEns45.time.dt.month[0:12]:
    print (i)
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.month==s))
    print(tmp1)
    i = i+1
    listds.append(tmp1)

0
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * time         (time) datetime64[ns] 1981-01-01 1982-01-01 ... 2100-01-01
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    dlyfrzthw    (realization, time, lat, lon) float64 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416,

## Loop to create all 12 dataframes

In [ ]:
i = 0
df45list = []
for ds in listds:
    print ("Dataset number: ", i, "--------------")
    perc45 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df45 = perc45.to_dataframe()
    df452 = df45.reset_index()
    df45=0
    df453 = df452.loc[df452["realization"] == 0].dropna()
    df452=0
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df453.groupby([df453.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp452 =  dfp.reset_index()
    dfp=0
    print ("Split in 3 different time periods")
    df45C = dfp452.copy()
    df45C[var+"_p10"] = round(df45C[var+"_p10"], 2)
    df45C[var+"_p50"] = round(df45C[var+"_p50"], 2)
    df45C[var+"_p90"] = round(df45C[var+"_p90"], 2)
    print ("Transformed in Celsiuds and rounded")
    Region1i45 = df45C.drop(columns=["realization"])
    df45C=0
    print ("pivoted")
    df45f = Region1i45.pivot_table(index=["lat","lon"], columns="time")
    Region1i45=0
    print ("DF pivoted")
    if i== 0: month = "january" 
    elif i == 1: month = "february"
    elif i == 2: month = "march"
    elif i == 3: month = "april"
    elif i == 4: month = "may"
    elif i == 5: month = "june"
    elif i == 6: month = "july"
    elif i == 7: month = "august"
    elif i == 8: month = "september"
    elif i == 9: month = "october"
    elif i == 10: month = "november"
    elif i == 11: month = "december"    
    print (month)         
    df45f.columns = [month + "_" + year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df45list.append(df45f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
january
Columns renamed
DF added to the list
Dataset number:  1 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
february
Columns renamed
DF added to the list
Dataset number:  2 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
march
Columns renamed
DF added to the list
Dataset number:  3 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
april
Columns renamed
DF added to the list
Dataset number:  4 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
may
Columns renamed
DF 

In [ ]:
df45list


In [ ]:
def merge_multiple_dfs(df_list, on):
    df_result = df_list[0]
    for df in df_list[1:]:
        df_result = df_result.merge(df, on=on)
    return df_result

In [ ]:
mkey = ["lat", "lon"]

In [ ]:
dfse45 = merge_multiple_dfs(df45list[0:12], on=mkey)

In [ ]:
dfse452 = dfse45.reset_index()
dfse45=0

In [ ]:
dfse452.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_monthly.feather")

In [ ]:
dfse452 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_monthly.feather")

In [ ]:
dfse453 = dfse452.drop(columns=["january_hist_rcp45_p10","january_hist_rcp45_p50", "january_hist_rcp45_p90", 
                               "february_hist_rcp45_p10", "february_hist_rcp45_p50", "february_hist_rcp45_p90", 
                               "march_hist_rcp45_p10","march_hist_rcp45_p50", "march_hist_rcp45_p90", 
                               "april_hist_rcp45_p10","april_hist_rcp45_p50", "april_hist_rcp45_p90",
                              "may_hist_rcp45_p10","may_hist_rcp45_p50", "may_hist_rcp45_p90", 
                               "june_hist_rcp45_p10", "june_hist_rcp45_p50", "june_hist_rcp45_p90", 
                               "july_hist_rcp45_p10","july_hist_rcp45_p50", "july_hist_rcp45_p90", 
                               "august_hist_rcp45_p10","august_hist_rcp45_p50", "august_hist_rcp45_p90",
                              "september_hist_rcp45_p10","september_hist_rcp45_p50", "september_hist_rcp45_p90", 
                               "october_hist_rcp45_p10", "october_hist_rcp45_p50", "october_hist_rcp45_p90", 
                               "november_hist_rcp45_p10","november_hist_rcp45_p50", "november_hist_rcp45_p90", 
                               "december_hist_rcp45_p10","december_hist_rcp45_p50", "december_hist_rcp45_p90"])
dfse453


In [ ]:
lisdfmoh45 = []
for month in monthlist: 
    dfmoh = dfse452[["lat", "lon", month+"_hist_rcp45_p50"]]
    print (month)
    lisdfmoh45.append(dfmoh)

# FOR RCP 85

## Subsetting arrays by periods of time with Xclim

In [ ]:
dsEns85= ens.create_ensemble(files85)
dsEns85.time.dt.month[0:12]
# Monthly example 30 y means
listds85 = []
i=0
for s in dsEns85.time.dt.month[0:12]:
    print (i)
    tmp1 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.month==s))
    print(tmp1)
    i = i+1
    listds85.append(tmp1)

## Loop to create all 12 dataframes

In [ ]:
i = 0
df85list = []
for ds in listds85:
    print ("Dataset number: ", i)
    perc85 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df85 = perc85.to_dataframe()
    df852 = df85.reset_index()
    df85=0
    df853 = df852.loc[df852["realization"] == 0].dropna()
    df852=0
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df853.groupby([df853.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp852 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df85C = dfp852.copy()
    dfp852=0
    df85C[var+"_p10"] = round(df85C[var+"_p10"], 2)
    df85C[var+"_p50"] = round(df85C[var+"_p50"], 2)
    df85C[var+"_p90"] = round(df85C[var+"_p90"], 2)
    print ("Transformed in Celsiuds and rounded")
    Region1i85 = df85C.drop(columns=["realization"])
    print ("pivoted")
    df85f = Region1i85.pivot_table(index=["lat","lon"], columns="time")
    Region1i85=0
    print ("DF pivoted")
    if i== 0: month = "january" 
    elif i == 1: month = "february"
    elif i == 2: month = "march"
    elif i == 3: month = "april"
    elif i == 4: month = "may"
    elif i == 5: month = "june"
    elif i == 6: month = "july"
    elif i == 7: month = "august"
    elif i == 8: month = "september"
    elif i == 9: month = "october"
    elif i == 10: month = "november"
    elif i == 11: month = "december"    
    print (month)
    df85f.columns = [month + "_" + year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df85list.append(df85f)
    print ("DF added to the list")
    i = i + 1
    

In [ ]:
df85list


In [ ]:
dfse85 = merge_multiple_dfs(df85list[0:12], on=mkey).reset_index()

In [ ]:
dfse85

In [ ]:
dfse85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_monthly.feather")

In [ ]:
dfse85 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_monthly.feather")

In [ ]:
dfse852 = dfse85.drop(columns=["january_hist_rcp85_p10","january_hist_rcp85_p50", "january_hist_rcp85_p90", 
                               "february_hist_rcp85_p10", "february_hist_rcp85_p50", "february_hist_rcp85_p90", 
                               "march_hist_rcp85_p10","march_hist_rcp85_p50", "march_hist_rcp85_p90", 
                               "april_hist_rcp85_p10","april_hist_rcp85_p50", "april_hist_rcp85_p90",
                              "may_hist_rcp85_p10","may_hist_rcp85_p50", "may_hist_rcp85_p90", 
                               "june_hist_rcp85_p10", "june_hist_rcp85_p50", "june_hist_rcp85_p90", 
                               "july_hist_rcp85_p10","july_hist_rcp85_p50", "july_hist_rcp85_p90", 
                               "august_hist_rcp85_p10","august_hist_rcp85_p50", "august_hist_rcp85_p90",
                              "september_hist_rcp85_p10","september_hist_rcp85_p50", "september_hist_rcp85_p90", 
                               "october_hist_rcp85_p10", "october_hist_rcp85_p50", "october_hist_rcp85_p90", 
                               "november_hist_rcp85_p10","november_hist_rcp85_p50", "november_hist_rcp85_p90", 
                               "december_hist_rcp85_p10","december_hist_rcp85_p50", "december_hist_rcp85_p90"])
dfse852


In [ ]:
lisdfmoh85 = []
for month in monthlist: 
    dfmoh = dfse85[["lat", "lon", month+"_hist_rcp85_p50"]]
    print (month)
    lisdfmoh85.append(dfmoh)

In [ ]:
lisdfmoh85

# Merge df45 and 85

# Extract hist

## Getting historic mean for each month

In [ ]:
### Function
def mean_month(month, dfse45, dfse85):
    print(month)
    dfw45 = dfse45[["lat", "lon", month+"_hist_rcp45_p50"]]
    dfw85 = dfse85[["lat", "lon", month+"_hist_rcp85_p50"]]
    dfsew = pd.merge(dfw45, dfw85, on=["lat","lon"])
    dfw45=0
    dfw85=0
    dfsew[month+"_hist_p50"] = round((dfsew[month+"_hist_rcp45_p50"]+ dfsew[month+"_hist_rcp85_p50"])/2, 2)
    dftgwh2 = dfsew.reset_index()
    dfsew=0
    dftgwh3 = dftgwh2[["lat", "lon", month+"_hist_p50"]]
    dftgwh2=0
    return (dftgwh3)


### All months historic mean

In [ ]:
listdfmo = []
i=0
for month in monthlist:
    print(month)
    dfm = mean_month(month, lisdfmoh45[i], lisdfmoh85[i])
    i = i+1
    listdfmo.append(dfm)

# Merge means with the 45 and 85 DF

In [ ]:
dftgseall = dfse852.merge(dfse452, on=mkey)
i=0
for df in listdfmo:
    dftgseall2 = dftgseall.merge(listdfmo[i], on=mkey)
    i = i +1


In [ ]:
dftgseall

In [ ]:
dfmop50 = merge_multiple_dfs(listdfmo[0:12], on=mkey)

In [ ]:
dfmop50

In [ ]:
dftgseall2 = dfse852.merge(dfse452, on=mkey)

In [ ]:
#dftgseall = dfse852.merge(dfse452, on=mkey).merge(dftfah, on=mkey).merge(dftsuh, on=mkey).merge(dftsph, on=mkey).merge(dftgwh, on=mkey)

In [ ]:
dftgseall2

In [ ]:
dftgseall3 = dftgseall2.merge(dfmop50, on=mkey)
dftgseall2=0

In [ ]:
dftgseall3.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_monthly.feather")

In [ ]:
dftgseall3

# Merge complete DF with Polygons for each spatial scale

## Dictionary - regions: column name, short name

In [ ]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

In [ ]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [ ]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [ ]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftgseall3, dfpolyshape, on=["lat","lon"])
    dfpolyshape = 0
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_"+var+"_monthly.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_"+var+"_monthly.json", driver="GeoJSON")
    dftp = 0
    gdf = 0